Для успешного выполнения этого задания вам предлагается “залезть в шкуру” трейдера.

 Имеются исторические данные по курсам валют (подсказка: ищите нужные данные в Example 2)  от ЦБ России. Пусть ваша команда выберет 5 валют (выберите любые валюты, которые вам интересны) и найдите ответы на следующие вопросы:
* для каждых последовательных 7 дней (например: 01.01.2000-07.01.2000, 02.01.2000-08.01.2000 и т. п.) найдите самую выгодную валюту - ту, относительно которой все остальные падали

* для каждого 30-дневного промежутка определите, на сколько процентов вы могли бы увеличить свой капитал за 30 дней, если бы знали курсы валют наперёд - считайте, что вечером вы можете перекладывать все ваши средства в одну или несколько валют, 
* найдите самый выгодный такой период
найдите наиболее и наименее коррелированные (по отношению друг к другу) валюты в каждом календарном году. Для этого посчитайте коэффициенты корреляции по данным за все 365/366 дней каждого года. Попробуйте выдвинуть гипотезы, почему случались те или иные взлёты/падения, в зависимости от политических и исторических событий.


In [ ]:
!pip install --quiet --upgrade --user nest_asyncio

In [ ]:
!pip install aiohttp

     |████████████████████████████████| 1.2MB 2.8MB/s 
     |████████████████████████████████| 266kB 20.8MB/s 
     |████████████████████████████████| 153kB 18.1MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=e63a7dda78979fc80cfd43bf6804caf5ba871abec32277bad45298df5b7636d8
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [ ]:
import requests
import asyncio
import datetime
import dateutil

import xml.etree.ElementTree as ET
import pandas as pd
import aiohttp
import nest_asyncio

In [ ]:
# Загрузка таблицы кодов валют

req = requests.get("http://www.cbr.ru/scripts/XML_valFull.asp")
root = ET.ElementTree(ET.fromstring(req.content)).getroot()


df_index = ['name', 'eng_name', 'nominal', 'code', 'ISO_char_code']   
currency_code = pd.DataFrame(columns=df_index)  
  
for element in root:  
    elements = [element[0].text, element[1].text, element[2].text, element[3].text.strip(' '), element[5].text]  
    currency_code = currency_code.append(pd.Series(elements, index=df_index), ignore_index=True) 

currency_code.head()

,name,eng_name,nominal,code,ISO_char_code
0,Австралийский доллар,Australian Dollar,1,R01010,AUD
1,Австрийский шиллинг,Austrian Shilling,1000,R01015,ATS
2,Азербайджанский манат,Azerbaijan Manat,1,R01020,AZN
3,Фунт стерлингов Соединенного королевства,British Pound Sterling,1,R01035,GBP
4,Ангольская новая кванза,Angolan new Kwanza,100000,R01040,AON


In [ ]:
'''
 Загрузка датафрейма с датой, курсом и номиналом
 по ISO_char_code
 '''

def load_currancy_by_char_code(char_code, start_date = '02/03/2001', end_date = '14/03/2020'):
    code = currency_code[currency_code['ISO_char_code']==char_code].code.values[0]
    url = "http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={}&date_req2={}&VAL_NM_RQ={}".format(start_date, end_date, code)
    req = requests.get(url)
    root = ET.ElementTree(ET.fromstring(req.content)).getroot()
    df_index = ['date', 'exchange_rate_{}'.format(char_code), 'nominal_{}'.format(char_code)]
    df = pd.DataFrame(columns=df_index)
    for element in root:
        elements = element.get('Date'), float(element[1].text.replace(',', '.')), element[0].text
        df = df.append(pd.Series(elements, index=df_index), ignore_index=True)
    return df


In [ ]:
load_currancy_by_char_code(char_code = 'AON', start_date = '02/03/2001', end_date = '30/09/2020').head()

,date,exchange_rate_AON,nominal_AON
0,01.04.2001,15.76,10
1,01.05.2001,15.80,10
2,01.06.2001,15.81,10
3,01.07.2001,14.68,10
4,01.08.2001,14.59,10


In [ ]:
def set_data(data, date):
  '''Добавляет данные в dataframe'''
  global df
  tree = ET.fromstring(data)
  for valute in tree.iter('Valute'):
    df = df.append({**{'date': date}, **{val.tag: val.text for val in valute}},
                   ignore_index=True)
  

In [ ]:
async def get_data(sem, url, session, date):
  '''Получение данных'''
  async with sem:
    try:
        r = await session.get(url, timeout=False)
    except (aiohttp.ClientConnectorError, aiohttp.ServerDisconnectedError, aiohttp.ClientOSError):
        print('\nОшибка подключения.')
        return
    if r.status == 200:
        set_data(await r.text(), date)


In [ ]:
async def upload_data(date_start, date_end=None):
  '''Асинхронная загрузка данных'''
  date_end = date_end or datetime.date.today()
  if date_end > datetime.date.today():
    raise ValueError('Дата окончания не может быть позже текущей')
  if date_start > datetime.date.today():
    raise ValueError('Дата начала не может быть позже текущей')
  async with aiohttp.ClientSession() as session:
    tasks = []
    sem = asyncio.Semaphore(25)
    current_date = date_start
    while current_date <= date_end:
      task = asyncio.ensure_future(get_data(sem, f'http://www.cbr.ru/scripts/XML_daily.asp?date_req={current_date.strftime("%d/%m/%Y")}', session, current_date))
      tasks.append(task)
      current_date += datetime.timedelta(days=1)
    await asyncio.gather(*tasks)



In [ ]:
df = pd.DataFrame()
nest_asyncio.apply()
loop = asyncio.get_event_loop()
loop.run_until_complete(upload_data(datetime.date.today()-dateutil.relativedelta.relativedelta(months=1), datetime.date.today()))
df

,CharCode,Name,Nominal,NumCode,Value,date
0,AUD,Австралийский доллар,1,036,"54,9221",2020-09-17
1,AZN,Азербайджанский манат,1,944,"44,1011",2020-09-17
2,GBP,Фунт стерлингов Соединенного королевства,1,826,"96,6644",2020-09-17
3,AMD,Армянских драмов,100,051,"15,4392",2020-09-17
4,BYN,Белорусский рубль,1,933,"29,0767",2020-09-17
...,...,...,...,...,...,...
1083,SEK,Шведских крон,10,752,"88,1553",2020-09-30
1084,CHF,Швейцарский франк,1,756,"86,2760",2020-09-30
1085,ZAR,Южноафриканских рэндов,10,710,"46,4064",2020-09-30
1086,KRW,Вон Республики Корея,1000,410,"68,1717",2020-09-30
